In [28]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-22,DL,DL,DL,544,DL,N344DN,544,11433,DTW,...,632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-07-04,AA,AA,AA,2623,AA,N929NN,2623,10721,BOS,...,1258,15.0,0.0,0.0,0.0,0.0,603.0,10.0,10.0,NaN
2,2019-06-27,AA,AA,AA,2036,AA,N807AW,2036,12953,LGA,...,544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-04-05,UA,UA_CODESHARE,UA,5306,OO,N870AS,5306,11292,DEN,...,649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-02-08,WN,WN,WN,548,WN,N953WN,548,10821,BWI,...,314,0.0,0.0,0.0,0.0,17.0,NaN,NaN,NaN,NaN


In [3]:
flights = flights[['fl_date','mkt_carrier','origin','dest'
                  ,'taxi_out','taxi_in','cancelled','crs_elapsed_time','arr_delay']]

In [4]:
#features and target
X = flights.loc[:,flights.columns!='cancelled']
y = flights[['cancelled']]

In [9]:
#lets create year, month and day
X['fl_date'] = pd.to_datetime(X['fl_date'],format='%Y-%m-%d')
X['year'] = pd.DatetimeIndex(X['fl_date']).year
X['month'] = pd.DatetimeIndex(X['fl_date']).month
X['day'] = pd.DatetimeIndex(X['fl_date']).day

In [11]:
X = X[['month','day','origin','dest','crs_elapsed_time']]

In [12]:
X= pd.get_dummies(X,columns=['origin','dest'])

In [29]:
#calculating the weights of class 0 to 1
class_0 = y.loc[y['cancelled'] == 0].value_counts().values[0]
class_1 = y.loc[y['cancelled'] == 1].value_counts().values[0]
weight = class_0/class_1
print('ratio of zeros to one:',weight)

ratio of zeros to one: 60.72839506172839


In [14]:
#Splitting into train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)

In [51]:
#giving the ratio as a weight to XGBClassifier
model = xgb.XGBClassifier(scale_pos_weight=80)
model.fit(X_train.values,y_train.values.reshape(1,-1)[0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=80, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [52]:
y_pred = model.predict(X_test.values)
print('Roc_Auc Score:',roc_auc_score(y_test,y_pred))
print('accuracy score: ',accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

Roc_Auc Score: 0.653395208980956
accuracy score:  0.6418888888888888


array([[56808, 31746],
       [  484,   962]], dtype=int64)